In [7]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Corpus

Dobbiamo chiaramente avere dei documenti. In questo caso, i documenti sono relativamente piccoli. Doc2VEc non ha un vero e proprio limite sulla lunghezza dei documenti che può vettorizzare. Ad ogni modo, ci sono alcune considerazioni pratiche da fare che potrebbero limitare la size effettiva dei documenti che si processano. Se si sta lavorando con documenti particolarmente lunghi, come libri interi o report piuttosto ricchi di informazione, potrebbe essere di beneficio **segmentare** i documenti in **parti logiche** o **capitoli** per poter vettorizzare queste parti in maniera indipendente.

Qualche dettaglio in più:
1. Documenti più lunghi richiedono più spazio in memoria e più potere computazionale per eseguire i processi. La complessità del modello aumenta con la **size del vocabolario** e con la **lunghezza dei documenti**.
2. Il parametro `window` nell'algoritmo definisce la maximum distance tra la parola corrent e la parola predetta in una frase. Per documenti molto lunghu, queto significa che parole molto distanti le une dalle altre non contribuiscono direttamente a imparare il contesto di una e dell'altra.
3. Il parametro `vector_size` definisce la diensionalità del vettore ottenuto dai documenti, e serve che questo vettore sia abbastanza lungo per catturare la **varianza** dei dati. Per dataset complessi o diversi in natura, una dimensionalità più alta è necessaria, ma questo aumenta i requirements computazionali richiesti al modello.
4. Per documenti estremamente lunghi si ha che la capacità del vettore che si calcola da essi nel rappresentare **accuratamente** il meaning semantico del documento intero può diminuire. Questo perché il modello fa una media del contesto di tutte le parti del documento in un singolo vettore di lunghezza fissa. Questo, di fatto, può portare ad una **perdita di informazione**, soprattutto per testi che sono complessi e sfaccettati nel loro significato.

In [8]:
# Sample documents
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Never jump over the lazy dog quickly.",
    "Bright vixens jump; dozy fowl quack.",
    "Quick wafting zephyrs vex bold Jim."
]

Serve ora **taggare** i documenti, fondamentale, poiché l'algoritmo richiede questo formato per i dati in ingresso. La libreria `gensim` mette a disposizione la classe `TaggedDocument`, specifica per Doc2Vec. Essa rappresenta un singolo documento in un formato che Doc2Vec può processare e comprendere.

Doc2Vec, diversamente da Word2Vec, richiede non solo la lista di parole della collezione, ma anche un identificatore, ossia un **tag**, per distinguere un documento dall'altro.  

Qui sotto vediamo che prima si splitta il testo in base agli spazi, e poi si creano gli oggetti TaggedDocument identificando come tag un progressivo.

In [10]:
# Tagging documents
tagged_documents = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]

In [11]:
tagged_documents

[TaggedDocument(words=['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog.'], tags=['0']),
 TaggedDocument(words=['Never', 'jump', 'over', 'the', 'lazy', 'dog', 'quickly.'], tags=['1']),
 TaggedDocument(words=['Bright', 'vixens', 'jump;', 'dozy', 'fowl', 'quack.'], tags=['2']),
 TaggedDocument(words=['Quick', 'wafting', 'zephyrs', 'vex', 'bold', 'Jim.'], tags=['3'])]

Vediamo un po' il significato dei parametri del training di Doc2Vec:


1. **tagged_documents**: The input for training, which is a list of `TaggedDocument` objects. Each `TaggedDocument` includes the words in the document and a unique tag associated with the document.

2. **vector_size=100**: This specifies the dimensionality of the document vectors. In this case, each document will be represented as a 100-dimensional vector. A higher dimension can capture more information but requires more data to train effectively and can increase computational complexity.

3. **window=5**: The maximum distance between the current and predicted word within a sentence. In other words, when considering a single word as input, the model will look at words 'window' distance away before and after the target word in the same sentence to provide context for learning.

4. **min_count=1**: This sets the minimum frequency count for words to be considered in the training. Words that appear fewer than `min_count` times across all documents are ignored. Setting it to 1 means that no words are disregarded due to their frequency.

5. **workers=4**: Determines the number of worker threads used to train the model, which can speed up training on multi-core machines. In this case, 4 worker threads will be used.

6. **epochs=40**: The number of iterations over the corpus during training. Higher epochs mean the model will have more opportunities to learn from the data, but it also increases the risk of overfitting if set too high.

In [16]:
# Train Doc2Vec model
model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4, epochs=40)

Ora ogni documento del training set viene anche trasformato in un vettore.

In [13]:
# Infer document vectors
doc_vectors = np.array([model.infer_vector(doc.words) for doc in tagged_documents])

In [18]:
# look at one vector
doc_vectors[0][0:10]

array([-0.0021444 , -0.00067057, -0.00221105,  0.00272681,  0.0024625 ,
       -0.00298162, -0.00207819,  0.00335603,  0.00413334,  0.00459327],
      dtype=float32)

In questo semplice esempio non si usa un vector database. Semplicemente, si riceve in input la query, la si trasforma attraverso lo stesso Doc2Vec model, e la si compara con **tutti** i vettori a nostra disposizione. Chiaramente questo è qualcosa che non faremmo se ci fosse un set di documenti molto grande. Il metodo seguente compara il vettore query con tutti i vettori documento a nostra disposizione, calcola la cosine similarity e ritorna la lista di documenti basandosi sullo score in maniera che i documenti più simili alla query, in base alla cosine similarity, sono in cima al ranking.

In [14]:
# Perform semantic search
def semantic_search(query, doc_vectors, model):
    query_vector = model.infer_vector(query.split())
    similarities = cosine_similarity([query_vector], doc_vectors)
    return np.argsort(similarities[0])[::-1]

In [15]:
# Example search
query = "Quick birds"
search_results = semantic_search(query, doc_vectors, model)
for idx in search_results:
    print(documents[idx])

Quick wafting zephyrs vex bold Jim.
Bright vixens jump; dozy fowl quack.
The quick brown fox jumps over the lazy dog.
Never jump over the lazy dog quickly.
